<a href="https://colab.research.google.com/github/ryannhally/Comps_Project_and_Essay/blob/main/CS_390_Pretrain_in_QU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets evaluate sacrebleu elotl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This beh

In [ ]:
# Sign into HuggingFace
from google.colab import userdata
userdata.get('HuggingFaceKey')

from huggingface_hub import notebook_login

notebook_login('HuggingFaceKey')

In [ ]:
from datasets import load_dataset
from datasets import Dataset

# Spanish - Quechua

In [ ]:
# Import tokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

tokenizer = AutoTokenizer.from_pretrained('google-t5/t5-small')
model = AutoModelForSeq2SeqLM.from_pretrained('google-t5/t5-small')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# Data collator
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
# Import dataset
qu_raw = load_dataset('somosnlp-hackathon-2022/spanish-to-quechua', split='train[:98%]')

README.md:   0%|          | 0.00/2.36k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/945 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/11.7M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.46M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.45M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/102747 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/12844 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/12843 [00:00<?, ? examples/s]

In [ ]:
# Check
print(qu_raw.num_rows)

100692


In [ ]:
# Remove null
qu_filtered = qu_raw.filter(lambda x:x['es'] is not None)

Filter:   0%|          | 0/100692 [00:00<?, ? examples/s]

In [ ]:
# Check
print(qu_filtered.num_rows)

100692


In [ ]:
# Preprocessing function
def qu_preprocess(example):

  inputs = ['translate Spanish to Quechua: ' + example for example in example['es']]
  targets = [example for example in example['qu']]
  model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)

  return model_inputs

In [ ]:
# Preprocess
qu_tokenized = qu_filtered.map(qu_preprocess, batched=True)

Map:   0%|          | 0/100692 [00:00<?, ? examples/s]

In [ ]:
# Check
print(qu_tokenized.num_rows)
print(qu_tokenized[1])
print(qu_tokenized[2])
print(qu_tokenized[50000])

100692
{'es': 'b) ¿Cómo lograban mantenerse estos siervos de tiempo completo?', 'qu': '(2) ¿Imaynatam tukuy tiemponkuwan Diosta serviq punta cristianokunaqa mantienekuqku?', 'input_ids': [13959, 5093, 12, 7227, 8019, 9, 10, 3, 115, 61, 3, 2, 254, 4922, 51, 32, 4303, 7093, 152, 388, 324, 277, 15, 259, 32, 7, 108, 49, 1621, 7, 20, 6177, 1167, 32, 3936, 32, 58, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [6499, 3, 2, 196, 13726, 29, 144, 265, 3, 17, 1598, 76, 63, 6177, 1167, 106, 2729, 3877, 2043, 3481, 9, 13482, 1824, 4930, 17, 9, 3, 2685, 7, 12572, 1825, 202, 9, 1824, 9, 388, 15945, 15, 2729, 1824, 2729, 58, 1]}
{'es': 'Cabe señalar que solo la Biblia explica de manera satisfactoria cuál es el origen de los muchos idiomas que conocemos hoy.', 'qu': 'Bibliallam allintapuni willawanchik imanasqam runakunaqa kunan tiempopi achka rimayniyoq kasqankuta.', 'input_ids': [13959, 5093, 12, 722

In [ ]:
# Split into train and text
qu_split = qu_tokenized.train_test_split(test_size=0.1)

In [ ]:
# Check
print(qu_split)

DatasetDict({
    train: Dataset({
        features: ['es', 'qu', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 90622
    })
    test: Dataset({
        features: ['es', 'qu', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 10070
    })
})


In [ ]:
# Hyperparemeters
training_args = Seq2SeqTrainingArguments(

    output_dir="ryannhally/t5-small-nah-qu",

    evaluation_strategy="steps",

    eval_steps=100,

    save_steps=500,

    learning_rate=2e-5,

    per_device_train_batch_size=16,

    per_device_eval_batch_size=16,

    weight_decay=0.01,

    num_train_epochs=40,

    fp16=True,

    push_to_hub=True

)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Seq2SeqTrainer(

    model=model,

    args=training_args,

    train_dataset=qu_split['train'],

    eval_dataset=qu_split['test'],

    tokenizer=tokenizer,

    data_collator=data_collator,


)

<ipython-input-26-126b61f02906>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss
100,No log,4.406532
200,No log,4.035861
300,No log,3.783377
400,No log,3.593869
500,4.277400,3.449988
600,4.277400,3.339786
700,4.277400,3.245337
800,4.277400,3.166990
900,4.277400,3.096342
1000,3.516800,3.039539


# Spanish - Nahuatl

In [ ]:
# Load model and tokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

tokenizer = AutoTokenizer.from_pretrained('ryannhally/t5-small-nah-qu')
model = AutoModelForSeq2SeqLM.from_pretrained('ryannhally/t5-small-nah-qu')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/20.7k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [ ]:
# Load axolotl dataset
axolotl_raw = load_dataset('somosnlp-hackathon-2022/Axolotl-Spanish-Nahuatl')
axolotl_filter = axolotl_raw.filter(lambda x:x['sp'] is not None)
axolotl = axolotl_filter.filter(lambda x:x['src'] != 'opus-bible-uedin')

README.md:   0%|          | 0.00/2.77k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/6.45M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20028 [00:00<?, ? examples/s]

Filter:   0%|          | 0/20028 [00:00<?, ? examples/s]

Filter:   0%|          | 0/20024 [00:00<?, ? examples/s]

In [ ]:
# Check
print(axolotl.num_rows)

{'train': 12203}


In [ ]:
# Normalize axolotl
import elotl.corpus
import elotl.nahuatl.orthography
n = elotl.nahuatl.orthography.Normalizer("sep")

translation = []
idx = []

i = 0
for example in axolotl['train']:
  idx.append(i)
  translation.append({'es': axolotl['train'][i]['sp'], 'na': n.normalize(axolotl['train'][i]['nah'])})
  i += 1

In [ ]:
# Load bible dataset
bible_raw = load_dataset('somosnlp-hackathon-2022/Axolotl-Spanish-Nahuatl')
bible_filter = bible_raw.filter(lambda x:x['sp'] is not None)
bible = bible_filter.filter(lambda x:x['src'] == 'opus-bible-uedin' )

Filter:   0%|          | 0/20024 [00:00<?, ? examples/s]

In [ ]:
# Normalize bible dataset
index = 0
for example in bible['train']:
  idx.append(i)
  translation.append({'es': bible['train'][index]['sp'], 'na': n.normalize(bible['train'][index]['nah'])})
  i +=1
  index +=1

In [ ]:
# Check
print(len(idx))
print(len(translation))
print(translation[0])
print(translation[13000])

20024
20024
{'es': 'Y así, cuando hizo su ofrenda de fuego, se sienta delante de los demás y una persona se queda junto a él.', 'na': 'au in ye yujki in on tlenamakak niman ye ik teixpan on motlalia se tlakatl itech mokaua.'}
{'es': ' "¡Ay de vosotros, escribas y fariseos, hipócritas!Porque edificáis los sepulcros de los profetas y adornáis los monumentos de los justos', 'na': "'¡lástima para nemejuamej temachtijkej ikan itlanauatil dios niman nemejuamej fariseos, nenomexayakyejkej!lástima, pampa nenkinchijchiuiliaj inmikakal on tiotlajtojkej, niman nenkitlatlajmachyotiaj ne kampa toktokej on yejuan kuajkualtin tlakamej katkaj."}


In [ ]:
# Make dictionary
data = { "id": idx, "translation": translation}

books = Dataset.from_dict(data)
books = books.train_test_split(test_size=0.1)

In [ ]:
# Check
print(books)
print(books['train'][0])
print(books['train'][18018] )
print(books['train'][500])

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 18021
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 2003
    })
})
{'id': 12548, 'translation': {'es': ' Pero os digo que uno mayor que el templo está aquí', 'na': 'tej, nemechijlia ika nikan nemi on yejuan más ueyixtikaj xken on ueyi tiopan.'}}
{'id': 13935, 'translation': {'es': ' tomarán serpientes en las manos, y si llegan a beber cosa venenosa, no les dañará.Sobre los enfermos pondrán sus manos, y sanarán.', 'na': 'tla kinkitskiskej kouamej, noso tla koniskej on veneno, xkinchiuilis daño.kintlaliskej inmauan impan on kokoxkej niman kimpajtiskej.'}}
{'id': 13450, 'translation': {'es': ' Ellos se burlaban de él.Pero él los sacó a todos y tomó al padre y a la madre de la niña y a los que estaban con él, y entró a donde estaba la niña', 'na': 'pero yejuamej okuetskilijkej.kemaj jesús okinkixtij ne kiauak on yejuan ompa nemiyaj, niman okuikak san itataj 

In [ ]:
# Define preprocessing function
source_lang = "es"

target_lang = "na"

prefix = "translate Spanish to Nahuatl: "


def preprocess_function(examples):

    inputs = [prefix + example[source_lang] for example in examples["translation"]]

    targets = [example[target_lang] for example in examples["translation"]]

    model_inputs = tokenizer(inputs, max_length=128, truncation=True)

    with tokenizer.as_target_tokenizer():

        labels = tokenizer(targets, max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [ ]:
# Preprocess
tokenized_books = books.map(preprocess_function, batched=True)

Map:   0%|          | 0/18021 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2003 [00:00<?, ? examples/s]

In [ ]:
# Check
print(tokenized_books)
print(tokenized_books['train'][0])
print(tokenized_books['train'][1])
print(tokenized_books['train'][200])
print(tokenized_books['train'][18000])

DatasetDict({
    train: Dataset({
        features: ['id', 'translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 18021
    })
    test: Dataset({
        features: ['id', 'translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2003
    })
})
{'id': 12548, 'translation': {'es': ' Pero os digo que uno mayor que el templo está aquí', 'na': 'tej, nemechijlia ika nikan nemi on yejuan más ueyixtikaj xken on ueyi tiopan.'}, 'input_ids': [13959, 5093, 12, 22885, 76, 144, 40, 10, 1915, 32, 3, 32, 7, 9799, 32, 238, 73, 32, 18176, 238, 3, 15, 40, 10301, 40, 32, 259, 2975, 3, 9, 2436, 2, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [3, 17, 15, 354, 6, 3, 9660, 15, 1436, 354, 40, 23, 9, 3, 5561, 3, 4953, 152, 3, 9660, 23, 30, 3, 63, 15, 2047, 152, 3, 28061, 3, 76, 15, 63, 2407, 4414, 9, 354, 3, 226, 2217, 30, 3, 76, 15, 63, 23, 3, 17, 23, 32, 2837, 5, 1]}
{'id': 705, 'tra

In [ ]:
# Hyperparemeters
training_args = Seq2SeqTrainingArguments(

    output_dir="ryannhally/t5-small-nah-qu-3",

    evaluation_strategy="steps",

    eval_steps=1000,

    save_steps=5000,

    learning_rate=2e-5,

    per_device_train_batch_size=16,

    per_device_eval_batch_size=16,

    weight_decay=0.01,

    num_train_epochs=280,

    save_total_limit=3,

    fp16=True,

    push_to_hub=True



)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Seq2SeqTrainer(

    model=model,

    args=training_args,

    train_dataset=tokenized_books['train'],

    eval_dataset=tokenized_books['test'],

    tokenizer=tokenizer,

    data_collator=data_collator,


)

<ipython-input-24-3245eda40cf6>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
1000,3.019300,2.738823
2000,2.871100,2.607239
3000,2.768600,2.510256
4000,2.686500,2.432945
5000,2.609000,2.374436
6000,2.555700,2.322866
7000,2.516300,2.279467
8000,2.473000,2.241102
9000,2.424200,2.206270
10000,2.386000,2.177217


TrainOutput(global_step=315560, training_loss=1.583278387081549, metrics={'train_runtime': 46403.6707, 'train_samples_per_second': 108.739, 'train_steps_per_second': 6.8, 'total_flos': 1.6615989704461517e+17, 'train_loss': 1.583278387081549, 'epoch': 280.0})